# Luis Tanoto (18/424191/PA/18296)
Pada tugas ini, saya akan melakukan prediksi terhadap headline berita yang ada apakah tergolong clickbait atau tidak, kemudian dilanjutkan dengan perhitungan evaluation metrics untuk menilai performa dari model yang telah dibuat. Namun, berbeda dari biasanya dimana model di-generate menggunakan library, kali ini tugas ini akan menggunakan cara manual untuk melakukan prediksi dengan menggunakan metode (Multinomial) Naive Bayes.

In [160]:
import pandas as pd #meng-import library pandas untuk meng-handle dataframe
import re # meng-import library re untuk men-generate regex

## 1. Training Phase

Pada tugas kali ini terdapat 2 dataset yaitu annotated_okezone.csv sebagai training set dan annotated_fimela.csv sebagai test set. Pada training set ini kita akan melakukan perhitungan probabilitas guna membantu prediksi class/label dari berita. Pertama-tama, agar dataframe lebih efektif, akan dilakukan drop terhadap kolom 'label'

In [161]:
train=pd.read_csv("annotated_okezone.csv")
train.head()

,title,label,label_score
0,Ini Penyebab Jamaah Tertipu Penggunaan Visa No...,clickbait,1
1,"Balita di Bogor Tewas dengan Luka Lebam, Ibu T...",non-clickbait,0
2,"Demi Keadilan, Pria Ini Habiskan Rp526 Juta La...",clickbait,1
3,Claudia Emanuela Santoso Harumkan Indonesia di...,non-clickbait,0
4,"Kalah Saing di Monza, Bottas Akui Ketangguhan ...",non-clickbait,0


In [162]:
train=train.drop("label",axis=1)
train

,title,label_score
0,Ini Penyebab Jamaah Tertipu Penggunaan Visa No...,1
1,"Balita di Bogor Tewas dengan Luka Lebam, Ibu T...",0
2,"Demi Keadilan, Pria Ini Habiskan Rp526 Juta La...",1
3,Claudia Emanuela Santoso Harumkan Indonesia di...,0
4,"Kalah Saing di Monza, Bottas Akui Ketangguhan ...",0
5,AFC Solidarity Cup Jadi Jalur Alternatif Timna...,0
6,"Bertitel Edisi Khusus, Vespa Sprint Carbon Dib...",0
7,"Finis Ke-11 di MotoGP San Marino, Zarco Cukup ...",0
8,Rodgers Akui Kesulitan Jaga Maddison dari Keja...,0
9,Titi DJ Borong Peserta Blind Audition The Voic...,1


Di bawah ini merupakan fungsi untuk melakukan text pre-processing terhadap dataset. Di fungsi tersebut berisi penghapusan pattern (seperti digit, punctuation) menggunakan regex dan juga proses stopwords removal. Namun, setelah melakukan beberapa kali percobaan, didapatkan hasil yang tidak lebih baik dari dataset tanpa dilakukan pre-processing, sehingga bagian ini tidak akan saya jalankan.

In [163]:
#stop words list
stop_words=set(['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 
            'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 
            'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 
            'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 
            'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 
            'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 
            'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 
            'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 
            'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 
            'seharusnya', 'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 'saja', 'toh', 'ya', 'walau', 'tolong', 
            'tentu', 'amat', 'apalagi', 'bagaimanapun'])

# pattern
pat_dig=r'[0-9]+'
pat_punc=r'[,\.%\"\'&:\[\]]+'
pattern=r'|'.join((pat_dig,pat_punc))

In [164]:
from nltk.tokenize import WordPunctTokenizer
token=WordPunctTokenizer()

def preprocessing(df):
    clean=[]
    for index,row in df.iterrows():
        pre_regex=re.sub(pat_punc,'',row.title)
        pre_lower=pre_regex.lower()
        pre_token=token.tokenize(pre_lower)
#         pre_filter=[w for w in pre_token if not w in stop_words]
#         clean.append(pre_lower.strip())
        clean.append((" ".join(pre_token).strip()))
    return clean

# proses=preprocessing(train)
# train.title=pd.DataFrame(proses)
# train

### 1.1. Create Vocabulary
Selanjutnya, untuk keperluan Naive Bayes nantinya, kita akan membangun vocabulary terlebih dahulu. Pertama-tama kita akan menggabungkan semua kalimat pada kolom 'title' dalam suatu list. Kemudian dengan fungsi vocab_count() kita akan membangun dictionary untuk menyimpan kata beserta frekuensi kemunculan. Selanjutnya list tadi akan displit untuk kemudian masuk ke perulangan untuk dihitung frekuensinya, jika kata tersebut belum ada di dictionary 'vocab_temp' maka akan dibuat dengan frekuensi 1, jika sudah dilakukan increment. Setelah itu return 'vocab_temp'.
Kemudian, karena pada tugas ini akan diambil hanya 1000 kata dengan frekuensi tertinggi, maka dilakukan sorting terlebih dahulu pada dictionary 'count' dan kemudian pada 'vocab_list' akan diambil 1000 kata pertama yang dijadikan sebagai vocabulary

In [165]:
bulk=" ".join(train['title'])

# fungsi untuk menghitung kata
def vocab_count(text):
    vocab_temp = dict()
    words = text.split()

    for w in words:
        if w in vocab_temp:
            vocab_temp[w] += 1
        else:
            vocab_temp[w] = 1

    return vocab_temp

count=vocab_count(bulk)
vocab={key: value for key, value in sorted(count.items(), reverse=True, key=lambda x: x[1])}
vocab_list=list(vocab)[:1000]
vocab_list

['di',
 'Ini',
 'dan',
 'yang',
 'ke',
 'Indonesia',
 'Tak',
 'KPK',
 ',',
 '2019',
 'Jadi',
 'dari',
 'untuk',
 'dengan',
 'hingga',
 'Liga',
 '5',
 'Jokowi',
 'Bisa',
 'UU',
 'Ibu',
 '3',
 'Liverpool',
 'DPR',
 'Revisi',
 'Baru',
 'Viral',
 'Karhutla',
 'Cukai',
 '4',
 'China',
 'Akan',
 'Kota',
 'MotoGP',
 'Ada',
 'Mobil',
 'Open',
 'Naik',
 'Hari',
 'Timnas',
 'Polisi',
 'saat',
 'Lebih',
 'Pertama',
 'Menteri',
 'Rumah',
 'Dunia',
 'Bikin',
 'Siap',
 'Kabut',
 'Tahun',
 'Tampil',
 'Pria',
 'Bakal',
 'Babak',
 'Ingin',
 'Tiga',
 'Hasil',
 'soal',
 'Asap',
 'Minta',
 'U-16',
 'Rokok',
 'Champions',
 'Pemerintah',
 'Ungkap',
 'Begini',
 'Man',
 'Pimpinan',
 'Anak',
 'dalam',
 '2020',
 'Orang',
 'Pakai',
 'Warga',
 'Setia',
 'Brimob',
 '2',
 'Masih',
 'Gagal',
 'San',
 'United',
 'Gara',
 'Punya',
 'Tidak',
 'Minyak',
 'Kembali',
 'karena',
 'Tersangka',
 'Inggris',
 'vs',
 'Mako',
 'Video',
 'Belum',
 'Harga',
 'Cara',
 'Kebakaran',
 'Napoli',
 'Akui',
 ',,',
 'Imam',
 'Harus',
 'pad

### 1.2. Estimate Parameter for Multinomial Naive Bayes Model

Proses selanjutnya adalah kita akan menghitung nilai conditional probability dari semua vocabulary.

Pertama-tama, kita akan menghitung banyak nilai 0 dan 1 pada kolom 'label_score' dan kemudian diubah menjadi dictionary serta disimpan pada 'count_label'

In [166]:
count_label=dict(train['label_score'].value_counts())
count_label

{1: 759, 0: 741}

Kemudian, dari nilai count_label tersebut kita akan menghitung nilai P(c=0) dan P(c=1). Pertama kita akan menghitung banyak dokumen pada training set dengan perulangan dan memasukkan pada variabel 'jumlah'.

Selanjutnya , akan dihitung P(c)=N(c)/N, dimana N(c) adalah banyaknya dokumen dengan kelas c dan N adalah variabel 'jumlah'

In [167]:
prob_label={}; jumlah=0
for i in count_label.keys():
    jumlah=jumlah+count_label[i]

for i in count_label.keys():
    prob_label[i]=count_label[i]/jumlah
prob_label

{1: 0.506, 0: 0.494}

Selanjutnya, kita akan menyiapkan suatu nested dictionary yang berisi 1000 vocabulary dari 'vocab_list' sebagai key dari dictionary pertama, dan untuk value dari dictionary pertama berisi dictionary kedua yang menunjukkan frekuensi/banyak kata di vocabulary pada masing-masing kelas, yaitu count(Wi | c=0) dan count(Wi | c=1). Namun sebelum menghitungnya, kita akan menginisialisasi dictionary seperti pada berikut untuk mengisi frekuensi semua kata pada setiap kelas adalah 0.

In [168]:
count_prob={i : {} for i in vocab_list}
for i in vocab_list:
    for j in count_label.keys():
        count_prob[i][j]=0
count_prob        
# for i in range(len(train)):
#     for j in vocab_list:
#         count_prob[j][str(train.iloc[i]['label_score'])]=count_prob[j][str(train.iloc[i]['label_score'])]+train.iloc[i]['title'].count(j)
# count_prob

{'di': {1: 0, 0: 0},
 'Ini': {1: 0, 0: 0},
 'dan': {1: 0, 0: 0},
 'yang': {1: 0, 0: 0},
 'ke': {1: 0, 0: 0},
 'Indonesia': {1: 0, 0: 0},
 'Tak': {1: 0, 0: 0},
 'KPK': {1: 0, 0: 0},
 ',': {1: 0, 0: 0},
 '2019': {1: 0, 0: 0},
 'Jadi': {1: 0, 0: 0},
 'dari': {1: 0, 0: 0},
 'untuk': {1: 0, 0: 0},
 'dengan': {1: 0, 0: 0},
 'hingga': {1: 0, 0: 0},
 'Liga': {1: 0, 0: 0},
 '5': {1: 0, 0: 0},
 'Jokowi': {1: 0, 0: 0},
 'Bisa': {1: 0, 0: 0},
 'UU': {1: 0, 0: 0},
 'Ibu': {1: 0, 0: 0},
 '3': {1: 0, 0: 0},
 'Liverpool': {1: 0, 0: 0},
 'DPR': {1: 0, 0: 0},
 'Revisi': {1: 0, 0: 0},
 'Baru': {1: 0, 0: 0},
 'Viral': {1: 0, 0: 0},
 'Karhutla': {1: 0, 0: 0},
 'Cukai': {1: 0, 0: 0},
 '4': {1: 0, 0: 0},
 'China': {1: 0, 0: 0},
 'Akan': {1: 0, 0: 0},
 'Kota': {1: 0, 0: 0},
 'MotoGP': {1: 0, 0: 0},
 'Ada': {1: 0, 0: 0},
 'Mobil': {1: 0, 0: 0},
 'Open': {1: 0, 0: 0},
 'Naik': {1: 0, 0: 0},
 'Hari': {1: 0, 0: 0},
 'Timnas': {1: 0, 0: 0},
 'Polisi': {1: 0, 0: 0},
 'saat': {1: 0, 0: 0},
 'Lebih': {1: 0, 0: 0},
 '

Langkah selanjutnya adalah membagi dataset 'train' ke dua buah dataset berdasarkan nilai 'label_score'-nya. Ini dilakukan agar saat proses perhitungan frekuensi kata, waktu run cell tidak terlalu lama. Dataset dibagi menjadi dataframe 'class_zero' untuk headline dengan label=0 dan 'class_one' untuk label=1. 

In [169]:
class_zero=train[train['label_score']==0]
class_zero

,title,label_score
1,"Balita di Bogor Tewas dengan Luka Lebam, Ibu T...",0
3,Claudia Emanuela Santoso Harumkan Indonesia di...,0
4,"Kalah Saing di Monza, Bottas Akui Ketangguhan ...",0
5,AFC Solidarity Cup Jadi Jalur Alternatif Timna...,0
6,"Bertitel Edisi Khusus, Vespa Sprint Carbon Dib...",0
7,"Finis Ke-11 di MotoGP San Marino, Zarco Cukup ...",0
8,Rodgers Akui Kesulitan Jaga Maddison dari Keja...,0
13,3 Mobil Listrik Ciptaan Mahasiswa UMS Siap Ber...,0
15,"BPK Temukan Pemborosan Rp275,19 Miliar oleh PLN",0
18,"Mane dan Salah Cetak Gol, Liverpool Tumpas New...",0


In [170]:
class_one=train[train['label_score']==1]
class_one

,title,label_score
0,Ini Penyebab Jamaah Tertipu Penggunaan Visa No...,1
2,"Demi Keadilan, Pria Ini Habiskan Rp526 Juta La...",1
9,Titi DJ Borong Peserta Blind Audition The Voic...,1
10,"Sukses di ,, Riri Muha Hadirkan Lagu",1
11,Memopulerkan Fesyen Ramah Lingkungan untuk Pri...,1
12,"Koleksi Mobil Menpora Imam Nahrawi, Termurah R...",1
14,"Lolos ke Babak Kedua, Rinov/Pitha Ingin Lebih ...",1
16,"Asyik Joget, Nikita Mirzani Tepergok Tak Kenakan",1
17,"Kemenhub Dapat Tambahan Anggaran Rp441,5 Milia...",1
21,"Jokowi Tinjau Penanganan Karhutla Riau, Wali K...",1


Berikutnya adalah proses perhitungan frekuensi kata dari setiap kelas. Konsepnya adalah akan dilakukan iterasi untuk setiap document/row, dilanjutkan dengan melakukan increment frekuensi pada setiap kata di vocabulary jika ditemukan kata pada 'title' yang cocok dengan kata yang ada di 'vocab_list'. Untuk penyocokkan kata, dimanfaatkan regex re.findall() dan negative lookbehind (<?<!\S) guna mendapatkan pola kata yang benar-benar sesuai dengan yang dicari (pada tugas ini, konteksnya adalah kata pada vocabulary). Karena ini sifatnya melakukan iterasi untuk penyocokkan 1000 vocabulary pada setiap document, proses ini memakan waktu kurang lebih 5-10 menit. Proses ini dilakukan pada dua dataset 'class_zero' dan 'class_one' dan akan menghasilkan nilai untuk count(Wi | c)

In [171]:
for i in range(len(class_zero)):
    for j in vocab_list:
        count_prob[j][0]=count_prob[j][0]+len(re.findall(r'(?<!\S)'+re.escape(j)+r'(?!\S)',class_zero.iloc[i]['title']))

In [172]:
for i in range(len(class_one)):
    for j in vocab_list:
        count_prob[j][1]=count_prob[j][1]+len(re.findall(r'(?<!\S)'+re.escape(j)+r'(?!\S)',class_one.iloc[i]['title']))

In [173]:
count_prob

{'di': {1: 197, 0: 228},
 'Ini': {1: 130, 0: 10},
 'dan': {1: 55, 0: 57},
 'yang': {1: 69, 0: 35},
 'ke': {1: 26, 0: 55},
 'Indonesia': {1: 30, 0: 48},
 'Tak': {1: 45, 0: 33},
 'KPK': {1: 19, 0: 59},
 ',': {1: 57, 0: 18},
 '2019': {1: 19, 0: 55},
 'Jadi': {1: 36, 0: 34},
 'dari': {1: 33, 0: 37},
 'untuk': {1: 32, 0: 37},
 'dengan': {1: 46, 0: 19},
 'hingga': {1: 27, 0: 20},
 'Liga': {1: 24, 0: 18},
 '5': {1: 36, 0: 4},
 'Jokowi': {1: 13, 0: 26},
 'Bisa': {1: 17, 0: 22},
 'UU': {1: 11, 0: 27},
 'Ibu': {1: 17, 0: 20},
 '3': {1: 27, 0: 7},
 'Liverpool': {1: 15, 0: 18},
 'DPR': {1: 4, 0: 29},
 'Revisi': {1: 9, 0: 24},
 'Baru': {1: 16, 0: 17},
 'Viral': {1: 32, 0: 1},
 'Karhutla': {1: 9, 0: 23},
 'Cukai': {1: 11, 0: 21},
 '4': {1: 22, 0: 10},
 'China': {1: 12, 0: 20},
 'Akan': {1: 10, 0: 21},
 'Kota': {1: 11, 0: 19},
 'MotoGP': {1: 7, 0: 22},
 'Ada': {1: 21, 0: 8},
 'Mobil': {1: 18, 0: 10},
 'Open': {1: 9, 0: 19},
 'Naik': {1: 12, 0: 14},
 'Hari': {1: 11, 0: 15},
 'Timnas': {1: 13, 0: 12},


Selanjutnya, kita akan menghitung total keseluruhan kata yang terdaftar dalam vocabulary untuk masing-masing kelas, yaitu nilai dari count(c=0) dan count(c=1) yang akan digunakan untuk menghitung probabilitas nantinya. Sama seperti sebelumnya, kita memanfaatkan dictionary 'count_words' untuk menyimpan nilainya. Untuk itu akan dilakukan iterasi untuk setiap label kelas. Guna menghitung jumlah, terdapat variabel 'jlh' sebagai penyimpan total kata per kelas. Setiap iterasi tersebut akan melakukan iterasi untuk setiap kata di vocabulary 'vocab_list'. Kemudian dengan memanfaatkan frekuensi 'count_prob' yang telah dihitung sebelumnya, akan dilakukan penjumlahan untuk setiap kata (yang ada di dalam 'vocab_list') kemudian disimpan dalam dictionary 'count_words' sebelum lanjut ke label berikutnya.

In [175]:
count_words={}
for j in count_label.keys():
    jlh=0
    for i in vocab_list:
        jlh=jlh+count_prob[i][j]
    count_words[j]=jlh
count_words

{1: 4291, 0: 4217}

Setelah mendapatkan semua elemen yang digunakan untuk perhitungan Multinomial Naive Bayes, kita akan langsung masuk ke rumus conditional probabilities. Sebelumnya, akan disiapkan terlebih dahulu nested dictionary 'cond_prob' untuk menyimpan probabilitas setiap kata untuk setiap kelasnya. 
Rumus conditional probabilities : 
P(Wi|C)=count(Wi|c)/count(c) dimana count(Wi|c) adalah frekuensi kemunculan kata Wi (pada vocab_list) untuk setiap kelas (count_prob) dan count(c) adalah total keseluruhan kata yang ada pada kelas c.

In [176]:
#Multinomial Naive Bayes
cond_prob={i : {} for i in vocab_list}
for i in count_prob.keys():
    for j in count_label.keys():
        cond_prob[i][j]=count_prob[i][j]/count_words[j]
cond_prob

{'di': {1: 0.04591004427872291, 0: 0.054066872184017074},
 'Ini': {1: 0.030295968305756232, 0: 0.0023713540431586438},
 'dan': {1: 0.01281752505243533, 0: 0.013516718046004269},
 'yang': {1: 0.016080167793055232, 0: 0.008299739151055252},
 'ke': {1: 0.006059193661151247, 0: 0.01304244723737254},
 'Indonesia': {1: 0.006991377301328362, 0: 0.01138249940716149},
 'Tak': {1: 0.010487065951992542, 0: 0.007825468342423524},
 'KPK': {1: 0.004427872290841296, 0: 0.013990988854635997},
 ',': {1: 0.013283616872523888, 0: 0.0042684372776855585},
 '2019': {1: 0.004427872290841296, 0: 0.01304244723737254},
 'Jadi': {1: 0.008389652761594034, 0: 0.008062603746739389},
 'dari': {1: 0.0076905150314611976, 0: 0.008774009959686982},
 'untuk': {1: 0.0074574691214169195, 0: 0.008774009959686982},
 'dengan': {1: 0.010720111862036821, 0: 0.004505572682001423},
 'hingga': {1: 0.006292239571195526, 0: 0.0047427080863172875},
 'Liga': {1: 0.00559310184106269, 0: 0.0042684372776855585},
 '5': {1: 0.0083896527615

### 1.3. Add-1 Smoothing

Kita sudah menghitung conditional probability untuk semua kata di vocabulary untuk setiap kelasnya pada 'cond_prob'. Namun, sebagaimana kita tahu ada beberapa kata di vocab_list yang probabilitas nya 0 dimana nanti saat melakukan perhitungan probabilitas pada test set akan menghasilkan nilai 0. Oleh karena itu, kita akan mengaplikasikan konsep add-1 smoothing pada nilai probabilitas yang sudah kita hitung agar setidaknya semua kata dianggap muncul minimal 1 kali.
Konsepnya sama seperti perhitungan sebelumnya, hanya rumusnya yang berbeda, yaitu :
P(Wi|C)=count(Wi|c)+1/count(c)+len(V) dimana len(V) adalah banyaknya vocabulary pada vocab_list. 

In [177]:
#Add-1 Smoothing
cond_prob={i : {} for i in vocab_list}
for i in count_prob.keys():
    for j in count_label.keys():
        cond_prob[i][j]=(count_prob[i][j]+1)/(count_words[j]+len(vocab_list))
cond_prob

{'di': {1: 0.037422037422037424, 0: 0.04389495878857581},
 'Ini': {1: 0.02475902475902476, 0: 0.0021084914701935977},
 'dan': {1: 0.010584010584010584, 0: 0.011117500479202607},
 'yang': {1: 0.01323001323001323, 0: 0.006900517538815411},
 'ke': {1: 0.005103005103005103, 0: 0.010734138393712862},
 'Indonesia': {1: 0.005859005859005859, 0: 0.009392371094498754},
 'Tak': {1: 0.008694008694008694, 0: 0.006517155453325666},
 'KPK': {1: 0.00378000378000378, 0: 0.011500862564692352},
 ',': {1: 0.010962010962010962, 0: 0.003641939812152578},
 '2019': {1: 0.00378000378000378, 0: 0.010734138393712862},
 'Jadi': {1: 0.006993006993006993, 0: 0.006708836496070539},
 'dari': {1: 0.006426006426006426, 0: 0.007283879624305156},
 'untuk': {1: 0.006237006237006237, 0: 0.007283879624305156},
 'dengan': {1: 0.008883008883008883, 0: 0.003833620854897451},
 'hingga': {1: 0.005292005292005292, 0: 0.0040253018976423235},
 'Liga': {1: 0.004725004725004725, 0: 0.003641939812152578},
 '5': {1: 0.0069930069930069

## 2. Test Phase

Kita sudah selesai menghitung conditional probability dari setiap kata di vocabulary kita dengan menggunakan konsep add-1 smoothing. Sekarang kita akan beralih ke dataset test untuk menguji performa dari perhitungan kita. Sama seperti training phase, kita akan menghapus kolom 'label' dan tidak melakukan text pre-processing karena akan menurunkan nilai akurasinya.

In [178]:
test=pd.read_csv("annotated_fimela.csv")
test.head()

,title,label,label_score
0,Lewat Seni Anak-Anak akan Tampil Percaya Diri ...,non-clickbait,0
1,"5 Manfaat Pilates untuk Ibu Hamil, Apa Saja?",clickbait,1
2,Pentingnya Sarapan dengan Makanan Padat untuk ...,clickbait,1
3,"Selalu Ingin Tahu, 5 Zodiak Ini Tidak Bisa Men...",clickbait,1
4,3 Jenis Diet untuk Organ Intim Lebih Sehat,clickbait,1


In [179]:
test=test.drop("label",axis=1)
test.head()

,title,label_score
0,Lewat Seni Anak-Anak akan Tampil Percaya Diri ...,0
1,"5 Manfaat Pilates untuk Ibu Hamil, Apa Saja?",1
2,Pentingnya Sarapan dengan Makanan Padat untuk ...,1
3,"Selalu Ingin Tahu, 5 Zodiak Ini Tidak Bisa Men...",1
4,3 Jenis Diet untuk Organ Intim Lebih Sehat,1


In [180]:
# proses_test=preprocessing(test)
# test.title=pd.DataFrame(proses_test)
# test

Selanjutnya kita akan mengekstrak kata-kata yang muncul di test set namun tidak ada di vocabulary 'vocab_list'. Ini dilakukan dengan menggabungkan semua kata di test set, dihitung frekuensinya dengan fungsi 'vocab_count' dan kemudian kata unik tersebut dicocokkan kemunculannya pada 'vocab_list' dan jika merupakan vocabulary baru akan disimpan di list 'vocab_test'. Karena kata-kata di 'vocab_test' merupakan kata yang tidak ada di 'vocab_list', maka probabilitasnya adalah 0 dan tentunya akan sangat mempengaruhi hasil prediksi nanti. Oleh karena itu, digunakan perhitungan add-1 smoothing pada kata-kata tersebut dan disimpan pada dictionary 'cond_prob_test'. Isi dari dictionary 'cond_prob_test' akan digabungkan dengan dictionary utama dari perhitungan 'cond_prob' dengan menggunakan method update()

In [181]:
bulk_test=" ".join(test['title'])
count_test=vocab_count(bulk_test)
vocab_test=[word for word in count_test.keys() if word not in vocab_list]
vocab_test

['Seni',
 'Anak-Anak',
 'akan',
 'Percaya',
 'Hingga',
 'Dewasa',
 'Manfaat',
 'Pilates',
 'Saja?',
 'Pentingnya',
 'Sarapan',
 'Padat',
 'Selalu',
 'Tahu,',
 'Menghargai',
 'Privasi',
 'Jenis',
 'Diet',
 'Organ',
 'Intim',
 'Sehat',
 'Move',
 'On,',
 'Tanda',
 'Mengajakmu',
 'Balikan',
 'Memakai',
 'Baju',
 'Dicuci',
 'Dulu',
 'Minum',
 'Secangkir',
 'Tiap',
 'Hari,',
 'Cokelat',
 'Panas',
 'Mencegah',
 'Diabetes',
 'Pertengkaran',
 'Pasangan',
 'Memperkuat',
 'Hubungan,',
 'Benarkah?',
 'Mineral?',
 'Faktanya',
 'Jatuh',
 'Sewajarnya',
 'Berujung',
 'Saling',
 'Menyakiti',
 'Tips',
 'Menyimpan',
 'Membeli',
 'Hemat',
 'Budget',
 'Tumis',
 'Pakcoy',
 'Bawang',
 'Putih',
 'Praktis',
 'Lelap,',
 'Melek',
 'Semalaman',
 'Mudah',
 'Bimbang,',
 'Dipengaruhi',
 'Menggunakan',
 'Kemiri',
 'Masakan',
 'Sedap',
 'Menikmati',
 'Pameran',
 'Lukisan',
 'Random',
 'Seniman',
 'Muda,',
 'Farrah',
 'Faustine',
 'Suka',
 'Kepo,',
 'Mengganggu',
 'Temannya!',
 'Sederhana',
 'Udang',
 'Bumbu',
 'Bantu'

In [182]:
#Add-1 Smoothing
cond_prob_test={i : {} for i in vocab_test}
for i in vocab_test:
    for j in count_label.keys():
        cond_prob_test[i][j]=1/(count_words[j]+len(vocab_list))
cond_prob_test

{'Seni': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Anak-Anak': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'akan': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Percaya': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Hingga': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Dewasa': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Manfaat': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Pilates': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Saja?': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Pentingnya': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Sarapan': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Padat': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Selalu': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Tahu,': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Menghargai': {1: 0.000189000189000189, 0: 0.00019168104274487253},
 'Privasi': {1: 0

In [183]:
cond_prob.update(cond_prob_test)
len(cond_prob)

3173

### 2.1. Count Probabilities of Each Document in Both Labels

Sebelum memprediksi label class sesuai perhitungan, kita akan menghitung terlebih dahulu probabilitas suatu document terhadap label c=0 dan c=1. Pertama kita akan memasukkan semua document tersebut ke dalam list 'temp_list' agar memudahkan kita dalam melakukan perhitungan. Di dalam list tersebut akan dibuat lagi list (nested list) untuk memecah setiap kata dalam document menjadi seperti token.

In [184]:
temp_list=[]
for j in range(len(test)):
    temp_list.append(test.iloc[j]['title'].split())
temp_list

[['Lewat',
  'Seni',
  'Anak-Anak',
  'akan',
  'Tampil',
  'Percaya',
  'Diri',
  'Hingga',
  'Dewasa'],
 ['5', 'Manfaat', 'Pilates', 'untuk', 'Ibu', 'Hamil,', 'Apa', 'Saja?'],
 ['Pentingnya', 'Sarapan', 'dengan', 'Makanan', 'Padat', 'untuk', 'Anak'],
 ['Selalu',
  'Ingin',
  'Tahu,',
  '5',
  'Zodiak',
  'Ini',
  'Tidak',
  'Bisa',
  'Menghargai',
  'Privasi',
  'Orang',
  'Lain'],
 ['3', 'Jenis', 'Diet', 'untuk', 'Organ', 'Intim', 'Lebih', 'Sehat'],
 ['Move',
  'On,',
  'Ini',
  '5',
  'Tanda',
  'Mantan',
  'Tak',
  'Akan',
  'Mengajakmu',
  'Balikan'],
 ['Mengenal', 'Bahaya', 'Memakai', 'Baju', 'Baru', 'Tanpa', 'Dicuci', 'Dulu'],
 ['Minum',
  'Secangkir',
  'Tiap',
  'Hari,',
  'Cokelat',
  'Panas',
  'Bisa',
  'Mencegah',
  'Diabetes'],
 ['Pertengkaran',
  'dengan',
  'Pasangan',
  'Bisa',
  'Memperkuat',
  'Hubungan,',
  'Benarkah?'],
 ['Apakah', 'Ada', 'Manfaat', 'Minum', 'Air', 'Mineral?', 'Ini', 'Faktanya'],
 ['Jatuh',
  'Cinta',
  'Sewajarnya',
  'Saja',
  'agar',
  'Tak',
 

Kita akan memulai perhitungan probabilitasnya. Konsepnya adalah kita melakukan perkalian probabilitas untuk setiap word di dalam document dan kemudian dikalikan dengan probabilitas dari label/classnya. Jadi kita melakukan iterasi untuk setiap kelasnya, kemudian untuk setiap kelas tersebut dilakukan iterasi lagi untuk akses ke setiap kata di 'temp_list'. Setelah itu, akan dilakukan perkalian probabilitas 'cond_prob' untuk setiap katanya hingga kata di document tersebut habis dan disimpan dalam 'prob_test'. Sebelum melanjutkan ke document berikutnya, nilai 'prob_test' akan dikalikan dengan 'prob_label'. Setelah itu nilai akhir 'prob_test' setiap document akan ditambahkan ke dalam list 'prob'. Nilai dari list 'prob' untuk setiap kelasnya akan dimasukkan ke test set dengan nama kolom 'prob_(0/1)'.

In [185]:
for i in count_label.keys():
    prob=[]
    for j in temp_list:
        prob_test=1
        for k in j:
            prob_test=prob_test*cond_prob[k][i]
            #print(prob_test,cond_prob[k][i])
        prob_test*=prob_label[i]
        prob.append(prob_test)
    #print(prob)
    test["prob_"+str(i)]=prob

test.head()

,title,label_score,prob_1,prob_0
0,Lewat Seni Anak-Anak akan Tampil Percaya Diri ...,0,5.605485e-33,6.212150e-32
1,"5 Manfaat Pilates untuk Ibu Hamil, Apa Saja?",1,6.518371e-25,7.183947e-27
2,Pentingnya Sarapan dengan Makanan Padat untuk ...,1,7.098838e-22,3.569396e-23
3,"Selalu Ingin Tahu, 5 Zodiak Ini Tidak Bisa Men...",1,1.510005e-36,7.914181e-39
4,3 Jenis Diet untuk Organ Intim Lebih Sehat,1,6.851141e-27,4.652461e-27


### 2.2. Predict Most Likely Class (CMAP)

Kita sudah selesai menghitung probabilitas setiap document untuk setiap kelasnya. Saatnya kita memprediksi CMAP dari setiap document. Untuk memudahkan saya menginisiasi list 'label' untuk menyimpan nilai dari prediksi.
Jadi, kita akan melakukan iterasi untuk setiap row/documentnya dan kemudian dibandingkan nilai antara probabilitas P(c=0|doc-x) dan P(c=1|doc-x). Jika nilai dari prob_1 > prob_0 maka, label akan ditambahkan dengan nilai 1, sebaliknya akan ditambahkan dengan nilai 0. Setelah itu dibuat kolom baru 'label_predicted' untuk menyimpan nilai list 'label'

In [186]:
label=[]
for index,row in test.iterrows():
    if row['prob_1']>row['prob_0']:
        label.append(1)
    else:
        label.append(0)

test['label_predicted']=label
test.head()

,title,label_score,prob_1,prob_0,label_predicted
0,Lewat Seni Anak-Anak akan Tampil Percaya Diri ...,0,5.605485e-33,6.212150e-32,0
1,"5 Manfaat Pilates untuk Ibu Hamil, Apa Saja?",1,6.518371e-25,7.183947e-27,1
2,Pentingnya Sarapan dengan Makanan Padat untuk ...,1,7.098838e-22,3.569396e-23,1
3,"Selalu Ingin Tahu, 5 Zodiak Ini Tidak Bisa Men...",1,1.510005e-36,7.914181e-39,1
4,3 Jenis Diet untuk Organ Intim Lebih Sehat,1,6.851141e-27,4.652461e-27,1


## 3. Evaluation Phase
### 3.1. Contigency Table Creation

Kita sudah mendapatkan hasil prediksi kita pada tahap test. Selanjutnya kita akan melakukan evaluasi terhadap perhitungan model Naive Bayes kita. Salah satu perhitungan evaluasi adalah dengan membangun confusion matrix atau contigency table. Unuk menghitung tersebut kita akan menghitung nilai dari TP, FP, FN, dan TN terlebih dahulu. Setelah mendapatkan nilainya, kita membangun confusion matrix dengan menggunakan nested list.

In [188]:
TP=0; FP=0; FN=0; TN=0;
for index,row in test.iterrows():
    if (row['label_score']==0 and row['label_predicted']==0):
        TP+=1
    elif (row['label_score']==0 and row['label_predicted']==1):
        FN+=1
    elif (row['label_score']==1 and row['label_predicted']==0):
        FP+=1
    else:
        TN+=1

confusion_matrix=[[TP,FP],[FN,TN]]
confusion_matrix

[[106, 45], [200, 349]]

### 3.2. Calculate Four Evaluation Metrics

Setelah memperoleh nilai TP, FP, FN, TN kita dapat membangun 4 metriks evaluasi yaitu accuracy, precision, recall, dan F1 Score dengan rumus dibawah.

In [189]:
accuracy=(TP+TN)/(TP+FP+FN+TN)
precision=TP/(TP+FP)
recall=TP/(TP+FN)
F1=2*precision*recall/(precision+recall)
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 Score: ",F1)

Accuracy:  0.65
Precision:  0.7019867549668874
Recall:  0.3464052287581699
F1 Score:  0.4638949671772428


### 3.3. Compare Microaveraging vs Macroaveraging Results

Tahap terakhir adalah menghitung nilai dari microaveraging dan macroaveraging. Sebelum memulai perhitungan, kita harus membangun tabel macroaverage dulu yaitu performa setiap kelasnya dan tabel microaverage.
Untuk macroaverage dibuat tabel 'class_zero' untuk performa dari label 0 dan 'class_one' untuk performa dari label 1. Untuk microaverage merupakan penjumlahan dari dari baris-kolom yang bersesuaian antara 'class_zero' dan 'class_one'.

In [190]:
class_zero=[[TP,FP],[FN,TN]]
class_one=[[TN,FN],[FP,TP]]
micro_table=[[TP+TN,FP+FN],[FN+FP,TN+TP]]
print("Class-0: ",class_zero)
print("Class-1: ",class_one)
print("Micro Avg. Table : ",micro_table)

Class-0:  [[106, 45], [200, 349]]
Class-1:  [[349, 200], [45, 106]]
Micro Avg. Table :  [[455, 245], [245, 455]]


Selanjutnya adalah perhitungan nilai microaveraging dan macroaveraging.
Nilai dari microaveraging maupun macroaveraging akan dihitung untuk 4 metrik evaluasi.
Perhitungan ini akan mengeluarkan hasil seperti di bawah.

In [191]:
#Accuracy
acc_zero=(class_zero[0][0]+class_zero[1][1])/(sum(class_zero[0])+sum(class_zero[1]))
acc_one=(class_one[0][0]+class_one[1][1])/(sum(class_one[0])+sum(class_one[1]))

macro_avg_acc=(acc_zero+acc_one)/2
micro_avg_acc=(micro_table[0][0]+micro_table[1][1])/(sum(micro_table[0])+sum(micro_table[1]))
print("Accuracy Class-0 : ",acc_zero,"\nAccuracy Class-1 : ",acc_one)
print("Macroaveraged Accuracy: ",macro_avg_acc)
print("Microaveraged Accuracy: ",micro_avg_acc)  

Accuracy Class-0 :  0.65 
Accuracy Class-1 :  0.65
Macroaveraged Accuracy:  0.65
Microaveraged Accuracy:  0.65


In [192]:
#Precision
pre_zero=(class_zero[0][0]/sum(class_zero[0]))
pre_one=(class_one[0][0]/sum(class_one[0]))

macro_avg_precision=(pre_zero+pre_one)/2
micro_avg_precision=((micro_table[0][0]/sum(micro_table[0])))
print("Precision Class-0 : ",pre_zero,"\nPrecision Class-1 : ",pre_one)
print("Macroaveraged Precision: ",macro_avg_precision)
print("Microaveraged Precision: ",micro_avg_precision)  

Precision Class-0 :  0.7019867549668874 
Precision Class-1 :  0.6357012750455373
Macroaveraged Precision:  0.6688440150062124
Microaveraged Precision:  0.65


In [193]:
#Recall
rec_zero=(class_zero[0][0]/(class_zero[0][0]+class_zero[1][0]))
rec_one=(class_one[0][0]/(class_one[0][0]+class_one[1][0]))

macro_avg_recall=(rec_zero+rec_one)/2
micro_avg_recall=((micro_table[0][0]/(micro_table[0][0]+micro_table[1][0])))
print("Recall Class-0 : ",rec_zero,"\nRecall Class-1 : ",rec_one)
print("Macroaveraged Recall: ",macro_avg_recall)
print("Microaveraged Recall: ",micro_avg_recall)  

Recall Class-0 :  0.3464052287581699 
Recall Class-1 :  0.8857868020304569
Macroaveraged Recall:  0.6160960153943134
Microaveraged Recall:  0.65


In [194]:
#F1-Score
p_zero=(class_zero[0][0]/sum(class_zero[0]))
p_one=(class_one[0][0]/sum(class_one[0]))
r_zero=(class_zero[0][0]/(class_zero[0][0]+class_zero[1][0]))
r_one=(class_one[0][0]/(class_one[0][0]+class_one[1][0]))
f1_zero=2*p_zero*r_zero/(p_zero+r_zero)
f1_one=2*p_one*r_one/(p_one+r_one)

macro_avg_f1=(f1_zero+f1_one)/2
micro_avg_f1=2*micro_avg_precision*micro_avg_recall/(micro_avg_precision+micro_avg_recall)
print("F1 Class-0 : ",f1_zero,"\nF1 Class-1 : ",f1_one)
print("Macroaveraged F1-Score: ",macro_avg_f1)
print("Microaveraged F1-Score: ",micro_avg_f1)  

F1 Class-0 :  0.4638949671772428 
F1 Class-1 :  0.7401908801696712
Macroaveraged F1-Score:  0.602042923673457
Microaveraged F1-Score:  0.65


## Analisis

Jika kita lihat dari hasil 4 evaluasi metriks diatas, semua microaveragenya sama yaitu 0.65. Namun, jika kita lihat satu per satu, nilai macroaveragenya bervariasi dan berbeda untuk setiap evaluasi metriks. Saya juga menampilkan perhitungan setiap evaluasi metriks per kelasnya sebelum dihitung rata-ratanya dalam macroaverage. Pada accuracy, semuanya sama bahkan dari perhitungan per kelasnya. Pada evaluation metrics lainnya seperti precision, recall, dan f1-score, antara macro-average dan micro-averagenya memiliki nilai yang hampir sama yaitu 60-70%. Tapi, jika kita lihat lebih dalam ke nilai setiap kelasnya, dapat kita lihat, misal pada class 0, presisinya sebesar 70% (dari 100 berita yang diprediksi tidak clickbait, 70 diantaranya benar) namun recall sebesar 34% (dari 100 berita yang tidak clickbait, 34 diantaranya benar). Ini dapat kita jelaskan bahwa model ini memberikan hasil yang baik ketika memprediksi class 1 (clickbait), namun tidak begitu baik ketika memprediksi class 0 (non clickbait). Ini juga sangat terlihat pada nilai macro- atau micro-averaged yang cukup kecil di label 0 dan cukup tinggi di label 1. <br>
Namun, secara keseluruhan dapat dilihat bahwa nilai macro- dan micro-averaged saling mendekati dan dapat disimpulkan bahwa model Multinomial Naive Bayes ini cukup baik dalam melakukan prediksi pada dataset ini meskipun hasilnya masih belum maksimal, yaitu sekitar 60-70%. Selain itu dari nilai macro- dan micro-averaged-nya, dapat kita simpulkan bahwa data ini termasuk balance data.